In [2]:
# %% [markdown]
# # Methodological Exploration: Comparative Analysis of Poverty Classifiers
# In this notebook, we evaluate how different algorithm families (Linear, Tree-based, Neural, and Bayesian) 
# model the complex socio-demographic and economic boundaries of poverty using the full "Oracle" dataset.

# %%
# 1. SETUP & DATA LOADING
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

print("📂 Loading Full Oracle Dataset...")

DATA_PROCESSED = os.path.join("..", "data", "processed")
oracle_path = os.path.join(DATA_PROCESSED, "df_person_oracle.csv")

if not os.path.exists(oracle_path):
    raise FileNotFoundError("❌ Oracle dataset missing. Run the Oracle generation step in the pipeline.")

df = pd.read_csv(oracle_path, dtype={'PUMA': str})

# Define Target and Weights
target = 'is_poor'
weights = 'Person_Weight'

# Auto-detect all features (excluding metadata/targets)
exclude_cols = [target, weights, 'PUMA', 'POVPIP'] 
features = [c for c in df.columns if c not in exclude_cols]

print(f"✨ Features detected ({len(features)}): {features}")

# Separate continuous vs categorical for the preprocessing pipeline
# (Age, Hours_Worked, and SNAP rates are continuous. Everything else is categorical)
continuous_candidates = ['Age', 'local_snap_claim_rate', 'Hours_Worked']
numeric_features = [c for c in continuous_candidates if c in features]
categorical_features = [c for c in features if c not in numeric_features]

# Cast categoricals to string so sklearn handles them strictly as categories
for c in categorical_features:
    df[c] = df[c].astype(str)

# Split the data
X = df[features]
y = df[target]
w = df[weights]

X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
    X, y, w, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Data loaded. Training size: {len(X_train)}")

📂 Loading Full Oracle Dataset...
✨ Features detected (6): ['Age', 'Sex_Code', 'Race_Code', 'local_snap_claim_rate', 'Education_Code', 'Employment_Status']
✅ Data loaded. Training size: 120578


In [4]:
# %%
# 2. THE MEGA BAKE-OFF
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, brier_score_loss

print("🍳 Preparing Preprocessing Pipelines...")

# Standardize inputs for fairness across model families
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Define the model zoo
models = {
    "Logistic Regression": Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(class_weight='balanced', max_iter=1000))
    ]),
    
    "Random Forest": Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=100, max_depth=10, class_weight='balanced', n_jobs=-1))
    ]),
    
    "Neural Network (MLP)": Pipeline([
        ('preprocessor', preprocessor),
        # 2 hidden layers, early stopping to prevent over-fitting
        ('classifier', MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', max_iter=500, early_stopping=True, random_state=42))
    ]),
    
    "Support Vector Machine": Pipeline([
        ('preprocessor', preprocessor),
        # LinearSVC wrapped in a calibrator to get Probabilities for AUC
        ('classifier', CalibratedClassifierCV(LinearSVC(class_weight='balanced', max_iter=2000, random_state=42), cv=3))
    ]),
    
    "LightGBM": lgb.LGBMClassifier(
        objective='binary', 
        metric='auc',
        is_unbalance=True, 
        n_estimators=300,
        learning_rate=0.05,
        n_jobs=-1,
        verbose=-1
    )
}

print(f"{'MODEL':<25} | {'AUC SCORE':<10} | {'BRIER (CALIB)':<10}")
print("-" * 55)

results = {}
for name, model in models.items():
    
    # LightGBM handles categories natively (bypasses the Pipeline)
    if name == "LightGBM":
        X_train_c = X_train.copy()
        X_test_c = X_test.copy()
        for c in categorical_features:
            X_train_c[c] = X_train_c[c].astype('category')
            X_test_c[c] = X_test_c[c].astype('category')
        model.fit(X_train_c, y_train, sample_weight=w_train)
        preds = model.predict_proba(X_test_c)[:, 1]
    
    # Sklearn models use the strict pipeline
    else:
        # MLP does not support sample_weight in scikit-learn
        if name == "Neural Network (MLP)":
            model.fit(X_train, y_train) 
        else:
            # Pass sample weights to the classifier step for the others
            model.fit(X_train, y_train, classifier__sample_weight=w_train)
            
        preds = model.predict_proba(X_test)[:, 1]
        
    # We still EVALUATE all models using the weights, so the test is fair
    auc = roc_auc_score(y_test, preds, sample_weight=w_test)
    brier = brier_score_loss(y_test, preds, sample_weight=w_test)
    
    results[name] = {'AUC': auc, 'Brier': brier}
    print(f"{name:<25} | {auc:.4f}     | {brier:.4f}")

🍳 Preparing Preprocessing Pipelines...
MODEL                     | AUC SCORE  | BRIER (CALIB)
-------------------------------------------------------
Logistic Regression       | 0.7619     | 0.2207
Random Forest             | 0.7722     | 0.2033
Neural Network (MLP)      | 0.7758     | 0.0792


/Users/zhenyuyue/miniforge3/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/zhenyuyue/miniforge3/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/zhenyuyue/miniforge3/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/zhenyuyue/miniforge3/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/zhenyuyue/miniforge3/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` wi

Support Vector Machine    | 0.7460     | 0.0825
LightGBM                  | 0.7852     | 0.1920


In [ ]:
# %%
# 3. BAYESIAN HIERARCHICAL MODELING (MrP)
import bambi as bmb
import arviz as az
import matplotlib.pyplot as plt

print("🧠 Building Bayesian Hierarchical Model...")

# 1. Take a 10% representative sample (Bayesian MCMC is computationally intense)
df_bayes = df.sample(frac=0.1, random_state=42, weights='Person_Weight')

# 2. Dynamically build the formula: Target ~ Feat1 + Feat2 + ... + (1|PUMA)
# (1|PUMA) tells the model to calculate a specific geographic offset for each PUMA
fixed_effects = " + ".join(features)
formula = f"{target} ~ {fixed_effects} + (1|PUMA)"

print(f"Formula: {formula}")

# 3. Initialize & Fit Model
model = bmb.Model(formula, df_bayes, family="bernoulli")

# MCMC Sampling - Added 'cores=1' to prevent the multiprocessing crash
print("Sampling posterior distribution (Sequentially)...")
results = model.fit(draws=1000, tune=1000, chains=2, cores=1, target_accept=0.9)

# 4. View Results
print("\n📊 Bayesian Model Fixed Effects:")
print(az.summary(results, var_names=features))

# Plot the geographic variances (Random Effects)
az.plot_forest(results, var_names=["1|PUMA"], combined=True)
plt.title("Level-2 Geographic Effects (PUMA Intercepts)")
plt.axvline(0, color='red', linestyle='--')
plt.show()

/Users/zhenyuyue/miniforge3/lib/python3.10/site-packages/arviz/__init__.py:50: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


🧠 Building Bayesian Hierarchical Model...
Formula: is_poor ~ Age + Sex_Code + Race_Code + local_snap_claim_rate + Education_Code + Employment_Status + (1|PUMA)
Sampling posterior distribution...


Modeling the probability that is_poor==1
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [Intercept, Age, Sex_Code, Race_Code, local_snap_claim_rate, Education_Code, Employment_Status, 1|PUMA_sigma, 1|PUMA_offset]


Output()

EOFError: 